In [1]:
#!/usr/bin/env python
# coding: utf-8

# # Step 03
# # Predict segmentations

import os
import os.path
import skimage.io
import skimage.morphology
import tensorflow as tf
import utils.dirtools
import utils.metrics
import utils.model_builder
from config import config_vars
from skimage import img_as_ubyte
import numpy as np
import glob
#import cv2

# # Configuration

#%%

# Partition of the data to make predictions (test or validation)
partition = "test"

experiment_name = 'MFGTMPcx7_170521130001'

config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

data_partitions = utils.dirtools.read_data_partitions(config_vars)

print(data_partitions[partition])


# In[4]:


# Device configuration

# Configuration to run on GPU
#import timeit

#start = timeit.default_timer()


#       inter_op_parallelism_threads=1)

# Configuration to run on GPU
'''
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = False
configuration.gpu_options.visible_device_list = "0"
session = tf.compat.v1.Session(config = configuration)
tf.compat.v1.keras.backend.set_session(session)
'''
#stop = timeit.default_timer()
#print('GPU set up: ', stop - start)

#with tf.compat.v1.Session() as sess:
#    devices = sess.list_devices()
#    print('devicessssssssssssssssssssssssssssssss',devices)
#global graph
#graph = tf.get_default_graph()


import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")
#from keras import backend as K
from keras.backend import clear_session



#####################################################################
def prediction_images(image_names):
    
    imagebuffer = skimage.io.imread_collection(image_names)

    images = imagebuffer.concatenate()

    

    dim1 = images.shape[1]
    dim2 = images.shape[2]

    images = images.reshape((-1, dim1, dim2,1))# config_vars["input_dimensions"]))

    # preprocess (assuming images are encoded as 8-bits in the preprocessing step)
    images = images /100#255
    
    with graph.as_default():
         predictions = model.predict(images, batch_size=1)
    
    print('after predicctionnnnnnnnnnnnnnnnnnnnnn')
    for i in range(len(imagebuffer.files)):
        filename_path = imagebuffer.files[i]
        filename = os.path.basename(filename_path)
        print(filename)
    
        probmap = predictions[i].squeeze()
        
    
    
        skimage.io.imsave(config_vars["probmap_out_dir"] + filename, (probmap).astype(float))
    
        pred = utils.metrics.probmap_to_pred(probmap, 
                                         config_vars["boundary_boost_factor"])


    
        label = utils.metrics.pred_to_label(pred,config_vars['cell_min_size'] )
    

    
        save_flag = skimage.io.imsave(config_vars["labels_out_dir"] + filename, (label).astype(int))

    

    return save_flag









####################################################################
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
from multiprocessing import cpu_count



##################################################################


with open(config_vars["path_files_MFGTMPcx7_170521130001"]) as image_list:
    image_names_all = [os.path.join(config_vars["MFGTMPcx7_170521130001_dir"], f.strip()) for f in image_list]

print('imaaaaaaaaaaaaaaaaageeeeeeeeeeeeeeeeeeeeeeeeees done')
clear_session()
global model
global graph

model = utils.model_builder.get_model_3_class(1104, 1104,1)
                                              #config_vars["input_dimensions"])
model.load_weights('model_14.hdf5')#config_vars["model_file"]) 

model._make_predict_function()
tf.Graph()
graph = tf.get_default_graph()


#print(model.summary())
#CPU_LIMIT=128
'''with ThreadPoolExecutor(min(CPU_LIMIT,cpu_count())) as executor:
    print('befoooooooooooooooooore submit function')
    futures=[executor.submit(prediction_images ,image_names_all[i_batch*16:(i_batch+1)*16]) for i_batch in range(384)]#range(len(image_names_all))] 

    for future in as_completed(futures):
        res=future.result()
        print('resssssssssssssuuuuuuuuuuuuuult: ',res)
'''
#test = [prediction_images(image_names_all[i_batch*2:(i_batch+1)*2]) for i_batch in range(2)]  ## use this if you do not want multi-threading
test = prediction_images(image_names_all)  ## use this if you do not want multi-threading







Using TensorFlow backend.


['MFGTMPcx7_170702000001_G14f03d0.png', 'MFGTMPcx7_170702090001_B22f15d0.png', 'MFGTMPcx7_170702090001_C08f14d0.png', 'MFGTMPcx7_170702090001_H04f01d0.png', 'MFGTMPcx7_170702090001_P07f14d0.png', 'MFGTMPcx7_170731090001_B14f09d0.png', 'MFGTMPcx7_170731090001_D11f13d0.png', 'MFGTMPcx7_170731090001_I12f05d0.png', 'MFGTMPcx7_170801050001_G02f01d0.png', 'MFGTMPcx7_170803210001_J12f29d0.png']
Default GPU Device:/device:GPU:0
imaaaaaaaaaaaaaaaaageeeeeeeeeeeeeeeeeeeeeeeeees done
Instructions for updating:
Colocations handled automatically by placer.


C:\Users\salma\OneDrive - Lund University\Segmentation_UNET_HoverNET\Test_code_gpu\plate_script\utils\model_builder.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
C:\Users\salma\OneDrive - Lund University\Segmentation_UNET_HoverNET\Test_code_gpu\plate_script\utils\model_builder.py:21: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
C:\Users\salma\OneDrive - Lund University\Segmentation_UNET_HoverNET\Test_code_gpu\plate_script\utils\model_builder.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
C:\Users\salma\OneDrive - Lund University\Segmentation_UNET_HoverNET\Test_code_gpu\plate_script\utils\mo

after predicctionnnnnnnnnnnnnnnnnnnnnn
MFGTMPcx7_170521130001_A04f12d0.png


C:\Users\salma\.conda\envs\TF_1\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\salma\.conda\envs\TF_1\lib\site-packages\skimage\io\_io.py:140: UserWarning: ./MFGTMPcx7_170521130001/out/segm/MFGTMPcx7_170521130001_A04f12d0.png is a low contrast image
  warn('%s is a low contrast image' % fname)


...

C:\Users\salma\.conda\envs\TF_1\lib\site-packages\skimage\io\_io.py:140: UserWarning: ./MFGTMPcx7_170521130001/out/segm/MFGTMPcx7_170521130001_P16f10d0.png is a low contrast image
  warn('%s is a low contrast image' % fname)


MFGTMPcx7_170521130001_P18f04d0.png


C:\Users\salma\.conda\envs\TF_1\lib\site-packages\skimage\io\_io.py:140: UserWarning: ./MFGTMPcx7_170521130001/out/segm/MFGTMPcx7_170521130001_P18f04d0.png is a low contrast image
  warn('%s is a low contrast image' % fname)


MFGTMPcx7_170521130001_P18f06d0.png


C:\Users\salma\.conda\envs\TF_1\lib\site-packages\skimage\io\_io.py:140: UserWarning: ./MFGTMPcx7_170521130001/out/segm/MFGTMPcx7_170521130001_P18f06d0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
